# Phone call and Google Analytics 📲

###### We load data from VOIP, clean it. We then load data from Goolge analytics, clean it. We then attach clinic phone numbers to Google analytics data, and proceed to join VOIP to Google Analytics data based on time of call and cliniced called

###  1 Loading voip data

In [18]:
import os
import pandas as pd
import time
import datetime

In [19]:
os.chdir(r"C:\Users\luisgarc0001\Documents\Projects\Marketing Analytics\VOIP")
path = os.getcwd()
files = os.listdir(path)


VOIP= pd.DataFrame()
data = pd.DataFrame()

for f in files:
      if f.endswith('.csv'):
        data = pd.read_csv(f,header=None,skiprows=1)
        data['filename'] = os.path.basename(f)
        VOIP = VOIP.append(data,ignore_index=True)   

VOIP.columns=["Call Time","Caller ID","Destination","Talking","file"]


In [20]:
### Clean VOIP file - Phone number and Date
VOIP['Phone'] = VOIP['Caller ID'].str.extract('.*\((.*)\).*') ##extract phone number from parenthesis
VOIP= VOIP[~VOIP['Caller ID'].str.contains("privacy",na=False)] ##filter privacy statement/ non numbers rows
VOIP['Phone'] =VOIP['Phone'].str.replace(r"[^0-9.]", "", regex= True) ##remove non-number character from Phone column
VOIP['Phone2']=VOIP.Phone.fillna(VOIP['Caller ID']) ##fill Phone 2 when Phone1 column is empty using Caller ID, otherwise use Phone1
VOIP['Phone2'] = VOIP['Phone2'].str.replace(r"[^0-9.]", "",  regex= True) ##remove non-numbers characters of this second phone option
VOIP['Phone_trim'] = VOIP['Phone2'].str[-8:] ##get last 8 digits
VOIP['Date'] = pd.to_datetime(VOIP['Call Time']) ##transform date 
VOIP['Date short']=VOIP['Date'].dt.strftime('%d/%m/%Y')  ##transform date to short

In [21]:
VOIP

,Call Time,Caller ID,Destination,Talking,file,Phone,Phone2,Phone_trim,Date,Date short
0,7/3/2021 10:06:42 AM,+31620167756 (0620167756),Q Wachtrij_DZH-ALmere (0501),00:00:07,almere.csv,0620167756,0620167756,20167756,2021-07-03 10:06:42,03/07/2021
1,7/3/2021 10:18:40 AM,+31653139994 (0653139994),Q Wachtrij_DZH-ALmere (0501),00:00:11,almere.csv,0653139994,0653139994,53139994,2021-07-03 10:18:40,03/07/2021
2,7/3/2021 10:23:30 AM,+31643407284 (0643407284),Q Wachtrij_DZH-ALmere (0501),00:00:06,almere.csv,0643407284,0643407284,43407284,2021-07-03 10:23:30,03/07/2021
3,7/3/2021 10:43:40 AM,+31687887196 (0687887196),Q Wachtrij_DZH-ALmere (0501),00:01:53,almere.csv,0687887196,0687887196,87887196,2021-07-03 10:43:40,03/07/2021
4,7/3/2021 10:59:20 AM,+31633399835 (0633399835),Q Wachtrij_DZH-ALmere (0501),00:00:52,almere.csv,0633399835,0633399835,33399835,2021-07-03 10:59:20,03/07/2021
...,...,...,...,...,...,...,...,...,...,...
53494,9/30/2021 1:11:39 PM,+31648057811 (0648057811),Q Q_Anicura-Zoetermeer (1515),00:00:19,Zoetermeer.csv,0648057811,0648057811,48057811,2021-09-30 13:11:39,30/09/2021
53495,9/30/2021 1:14:46 PM,+31648057811 (0648057811),Q Q_Anicura-Zoetermeer (1515),00:00:22,Zoetermeer.csv,0648057811,0648057811,48057811,2021-09-30 13:14:46,30/09/2021
53496,9/30/2021 1:26:26 PM,+31683325130 (0683325130),Q Q_Anicura-Zoetermeer (1515),00:00:04,Zoetermeer.csv,0683325130,0683325130,83325130,2021-09-30 13:26:26,30/09/2021
53497,9/30/2021 1:41:19 PM,+31611746628 (0611746628),Q Q_Anicura-Zoetermeer (1515),00:00:17,Zoetermeer.csv,0611746628,0611746628,11746628,2021-09-30 13:41:19,30/09/2021


In [62]:
VOIP.to_excel(r"C:\Users\luisgarc0001\Documents\Projects\Marketing Analytics\voip.xlsx")

### 2 Loading Google analytics data

In [23]:
website= pd.read_csv(r"C:\Users\luisgarc0001\Documents\Projects\Marketing Analytics\anicura phone call timestamps - Sheet5.csv")
website['Phone'] = website['Phone number'].str.replace(r"[^0-9.]", "",regex= True) ##extract phone number from parenthesis
website['Date'] = pd.to_datetime(website['Timestamp']) ##transform date 
website['Phone_trim'] = website['Phone'].str[-8:] ##get last 8 digits
website['RowIdentifier']=website['Timestamp'].astype(str)+website['Google Analytics Client ID'].astype(str) ##create unique id per row
website['Date short']=website['Date'].dt.strftime('%d/%m/%Y')##transform date to short

In [24]:
website

,Phone number,Timestamp,Google Analytics Client ID,Phone,Date,Phone_trim,RowIdentifier,Date short
0,023 - 563 53 11,2021-09-29 23:23:00,2.107742e+09,0235635311,2021-09-29 23:23:00,35635311,2021-09-29 23:23:002107742118.0,29/09/2021
1,0186 - 612 066,2021-09-29 20:58:00,1.684560e+08,0186612066,2021-09-29 20:58:00,86612066,2021-09-29 20:58:00168455995.2,29/09/2021
2,078 - 693 00 55,2021-09-29 19:24:00,9.119373e+08,0786930055,2021-09-29 19:24:00,86930055,2021-09-29 19:24:00911937251.2,29/09/2021
3,050 - 409 46 65,2021-09-29 19:23:00,3.070874e+08,0504094665,2021-09-29 19:23:00,04094665,2021-09-29 19:23:00307087352.2,29/09/2021
4,023 - 538 28 18,2021-09-29 19:11:00,8.746029e+08,0235382818,2021-09-29 19:11:00,35382818,2021-09-29 19:11:00874602943.2,29/09/2021
...,...,...,...,...,...,...,...,...
3322,0515 %E2%80%93 412 427,2021-08-31 08:33:00,1.906682e+07,051528093412427,2021-08-31 08:33:00,93412427,2021-08-31 08:33:0019066819.16,31/08/2021
3323,013 - 540 02 15,2021-08-31 08:30:00,4.623473e+08,0135400215,2021-08-31 08:30:00,35400215,2021-08-31 08:30:00462347348.2,31/08/2021
3324,071 %E2%80%93 51 56 16 1,2021-08-31 08:09:00,1.513127e+08,071280935156161,2021-08-31 08:09:00,35156161,2021-08-31 08:09:00151312731.2,31/08/2021
3325,0186 - 612 066,2021-08-31 07:50:00,2.595708e+08,0186612066,2021-08-31 07:50:00,86612066,2021-08-31 07:50:00259570838.2,31/08/2021


### 3 Load the Phone number of clinics as per Stanley

In [25]:
mapin= pd.read_excel(r"C:\Users\luisgarc0001\Documents\Projects\Marketing Analytics\Telefoonnummers.xlsx")

In [26]:
mapin['Telefoonnummer'] = mapin['Telefoonnummer'].astype(str)
mapin['Phone_trim'] = mapin['Telefoonnummer'].str[-8:]

In [27]:
mapin

,Telefoonnummer,Clinic,File name,Phone_trim
0,31223620006,Anicura Den Helder,den helder.csv,23620006
1,31850200080,Anicura Den Helder,den helder.csv,50200080
2,31102950530,AniCura Dierenkliniek Hoogvliet,Hoogvliet.csv,02950530
3,31104720540,AniCura Dierenkliniek Hoogvliet,Hoogvliet.csv,04720540
4,31181690055,AniCura Dierenkliniek Hoogvliet,Hoogvliet.csv,81690055
...,...,...,...,...
110,31207400697,Anicura MCD,MCD.csv,07400697
111,31207400698,Anicura MCD,MCD.csv,07400698
112,31207400699,Anicura MCD,MCD.csv,07400699
113,31850203628,Anicura MCD,MCD.csv,50203628


###### Now merge 3 and 2, called clinics with their names

In [28]:
website = pd.merge(website, mapin, left_on="Phone_trim", right_on="Phone_trim")

In [29]:
website

,Phone number,Timestamp,Google Analytics Client ID,Phone,Date,Phone_trim,RowIdentifier,Date short,Telefoonnummer,Clinic,File name
0,0183 - 30 17 08,2021-09-29 17:16:00,1.500689e+09,0183301708,2021-09-29 17:16:00,83301708,2021-09-29 17:16:001500688525.0,29/09/2021,31183301708,AniCura Sleeuwijk,sleeuwijk.csv
1,0183 - 30 17 08,2021-09-29 11:25:00,1.158907e+09,0183301708,2021-09-29 11:25:00,83301708,2021-09-29 11:25:001158906979.0,29/09/2021,31183301708,AniCura Sleeuwijk,sleeuwijk.csv
2,0183 - 30 17 08,2021-09-28 17:49:00,1.164044e+09,0183301708,2021-09-28 17:49:00,83301708,2021-09-28 17:49:001164044300.0,28/09/2021,31183301708,AniCura Sleeuwijk,sleeuwijk.csv
3,0183 - 30 17 08,2021-09-28 16:37:00,6.579564e+08,0183301708,2021-09-28 16:37:00,83301708,2021-09-28 16:37:00657956394.2,28/09/2021,31183301708,AniCura Sleeuwijk,sleeuwijk.csv
4,0183 - 30 17 08,2021-09-28 11:39:00,2.086221e+09,0183301708,2021-09-28 11:39:00,83301708,2021-09-28 11:39:002086221143.0,28/09/2021,31183301708,AniCura Sleeuwijk,sleeuwijk.csv
...,...,...,...,...,...,...,...,...,...,...,...
377,020 - 740 06 00,2021-09-01 08:59:00,1.535522e+07,0207400600,2021-09-01 08:59:00,07400600,2021-09-01 08:59:0015355218.16,01/09/2021,31207400600,Anicura MCD,MCD.csv
378,+31653512253,2021-09-24 11:50:00,2.053012e+09,31653512253,2021-09-24 11:50:00,53512253,2021-09-24 11:50:002053011837.0,24/09/2021,31653512253,AniCura Dierenkliniek Hoogvliet,Hoogvliet.csv
379,+31653512253,2021-09-23 12:04:00,1.680149e+08,31653512253,2021-09-23 12:04:00,53512253,2021-09-23 12:04:00168014923.2,23/09/2021,31653512253,AniCura Dierenkliniek Hoogvliet,Hoogvliet.csv
380,+31653512253,2021-09-06 15:25:00,9.316825e+08,31653512253,2021-09-06 15:25:00,53512253,2021-09-06 15:25:00931682526.2,06/09/2021,31653512253,AniCura Dierenkliniek Hoogvliet,Hoogvliet.csv


### Join Call Google with VOIP data

###### We join inner because we want matches that appear in both files. We join based on file, which is the clinic called, and Date short, which is the date of call. In essence, we are joining all calls to a clinic in a given day across data sources

In [30]:
Google_Voip = pd.merge(VOIP, website,  how='inner', left_on=["file",'Date short'], right_on=["File name",'Date short'])


In [31]:
Google_Voip

,Call Time,Caller ID,Destination,Talking,file,Phone_x,Phone2,Phone_trim_x,Date_x,Date short,Phone number,Timestamp,Google Analytics Client ID,Phone_y,Date_y,Phone_trim_y,RowIdentifier,Telefoonnummer,Clinic,File name
0,8/31/2021 9:00:56 AM,+31611333546 (0611333546),Q Wachtrij_DZH-ALmere (0501),00:00:06,almere.csv,0611333546,0611333546,11333546,2021-08-31 09:00:56,31/08/2021,036 - 536 96 00,2021-08-31 16:00:00,1.579120e+09,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv
1,8/31/2021 9:04:27 AM,+31634540500 (0634540500),Q Wachtrij_DZH-ALmere (0501),00:07:54,almere.csv,0634540500,0634540500,34540500,2021-08-31 09:04:27,31/08/2021,036 - 536 96 00,2021-08-31 16:00:00,1.579120e+09,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv
2,8/31/2021 9:05:12 AM,+31615882775 (0615882775),Q Wachtrij_DZH-ALmere (0501),unanswered,almere.csv,0615882775,0615882775,15882775,2021-08-31 09:05:12,31/08/2021,036 - 536 96 00,2021-08-31 16:00:00,1.579120e+09,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv
3,8/31/2021 9:06:08 AM,+31642153759 (0642153759),Q Wachtrij_DZH-ALmere (0501),unanswered,almere.csv,0642153759,0642153759,42153759,2021-08-31 09:06:08,31/08/2021,036 - 536 96 00,2021-08-31 16:00:00,1.579120e+09,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv
4,8/31/2021 9:06:55 AM,+31642153759 (0642153759),Q Wachtrij_DZH-ALmere (0501),00:13:57,almere.csv,0642153759,0642153759,42153759,2021-08-31 09:06:55,31/08/2021,036 - 536 96 00,2021-08-31 16:00:00,1.579120e+09,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54045,9/29/2021 4:43:13 PM,+31611865036 (0611865036),Q Q_Anicura-Zoetermeer (1515),00:00:10,Zoetermeer.csv,0611865036,0611865036,11865036,2021-09-29 16:43:13,29/09/2021,079 - 346 02 14,2021-09-29 13:22:00,1.010638e+09,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv
54046,9/29/2021 5:12:57 PM,+31642123392 (0642123392),Q Q_Anicura-Zoetermeer (1515),00:00:21,Zoetermeer.csv,0642123392,0642123392,42123392,2021-09-29 17:12:57,29/09/2021,079 - 346 02 14,2021-09-29 13:22:00,1.010638e+09,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv
54047,9/29/2021 6:42:01 PM,+31648444026 (0648444026),Q Q_Anicura-Zoetermeer (1515),00:00:07,Zoetermeer.csv,0648444026,0648444026,48444026,2021-09-29 18:42:01,29/09/2021,079 - 346 02 14,2021-09-29 13:22:00,1.010638e+09,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv
54048,9/29/2021 7:43:48 PM,+31640594932 (0640594932),Q Q_Anicura-Zoetermeer (1515),00:00:03,Zoetermeer.csv,0640594932,0640594932,40594932,2021-09-29 19:43:48,29/09/2021,079 - 346 02 14,2021-09-29 13:22:00,1.010638e+09,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv


In [33]:
#Clean and remove not good matches.
###Find time difference between call in VOIP and Google Analytics data, find this in Hours, and Minutes
Google_Voip["timejumps"]=(Google_Voip["Date_x"]- Google_Voip["Date_y"]).apply(str) 
Google_Voip["hours"]=(Google_Voip["Date_x"]- Google_Voip["Date_y"]).astype('timedelta64[h]')
Google_Voip["min"]=(Google_Voip["Date_x"]- Google_Voip["Date_y"]).astype('timedelta64[m]')
Google_Voip["timejumps"]=Google_Voip["timejumps"].apply(lambda x: x.split(' ')[0])
Google_Voip["timejumps"]=Google_Voip["timejumps"].apply(int)



In [34]:
Google_Voip

,Call Time,Caller ID,Destination,Talking,file,Phone_x,Phone2,Phone_trim_x,Date_x,Date short,...,Phone_y,Date_y,Phone_trim_y,RowIdentifier,Telefoonnummer,Clinic,File name,timejumps,hours,min
0,8/31/2021 9:00:56 AM,+31611333546 (0611333546),Q Wachtrij_DZH-ALmere (0501),00:00:06,almere.csv,0611333546,0611333546,11333546,2021-08-31 09:00:56,31/08/2021,...,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv,-1,-7.0,-420.0
1,8/31/2021 9:04:27 AM,+31634540500 (0634540500),Q Wachtrij_DZH-ALmere (0501),00:07:54,almere.csv,0634540500,0634540500,34540500,2021-08-31 09:04:27,31/08/2021,...,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv,-1,-7.0,-416.0
2,8/31/2021 9:05:12 AM,+31615882775 (0615882775),Q Wachtrij_DZH-ALmere (0501),unanswered,almere.csv,0615882775,0615882775,15882775,2021-08-31 09:05:12,31/08/2021,...,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv,-1,-7.0,-415.0
3,8/31/2021 9:06:08 AM,+31642153759 (0642153759),Q Wachtrij_DZH-ALmere (0501),unanswered,almere.csv,0642153759,0642153759,42153759,2021-08-31 09:06:08,31/08/2021,...,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv,-1,-7.0,-414.0
4,8/31/2021 9:06:55 AM,+31642153759 (0642153759),Q Wachtrij_DZH-ALmere (0501),00:13:57,almere.csv,0642153759,0642153759,42153759,2021-08-31 09:06:55,31/08/2021,...,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv,-1,-7.0,-414.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54045,9/29/2021 4:43:13 PM,+31611865036 (0611865036),Q Q_Anicura-Zoetermeer (1515),00:00:10,Zoetermeer.csv,0611865036,0611865036,11865036,2021-09-29 16:43:13,29/09/2021,...,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,3.0,201.0
54046,9/29/2021 5:12:57 PM,+31642123392 (0642123392),Q Q_Anicura-Zoetermeer (1515),00:00:21,Zoetermeer.csv,0642123392,0642123392,42123392,2021-09-29 17:12:57,29/09/2021,...,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,3.0,230.0
54047,9/29/2021 6:42:01 PM,+31648444026 (0648444026),Q Q_Anicura-Zoetermeer (1515),00:00:07,Zoetermeer.csv,0648444026,0648444026,48444026,2021-09-29 18:42:01,29/09/2021,...,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,5.0,320.0
54048,9/29/2021 7:43:48 PM,+31640594932 (0640594932),Q Q_Anicura-Zoetermeer (1515),00:00:03,Zoetermeer.csv,0640594932,0640594932,40594932,2021-09-29 19:43:48,29/09/2021,...,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,6.0,381.0


In [35]:
###Now filter for only when the hour difference between one systemm and the other is less than 60 min
Google_Voip=Google_Voip[Google_Voip['hours']==0]


In [36]:
Google_Voip

,Call Time,Caller ID,Destination,Talking,file,Phone_x,Phone2,Phone_trim_x,Date_x,Date short,...,Phone_y,Date_y,Phone_trim_y,RowIdentifier,Telefoonnummer,Clinic,File name,timejumps,hours,min
34,8/31/2021 4:00:23 PM,+31623083592 (0623083592),Q Wachtrij_DZH-ALmere (0501),00:01:16,almere.csv,0623083592,0623083592,23083592,2021-08-31 16:00:23,31/08/2021,...,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv,0,0.0,0.0
35,8/31/2021 4:07:14 PM,+31615229600 (0615229600),Q Wachtrij_DZH-ALmere (0501),00:00:19,almere.csv,0615229600,0615229600,15229600,2021-08-31 16:07:14,31/08/2021,...,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv,0,0.0,7.0
36,8/31/2021 4:10:28 PM,+31611014349 (0611014349),Q Wachtrij_DZH-ALmere (0501),00:00:05,almere.csv,0611014349,0611014349,11014349,2021-08-31 16:10:28,31/08/2021,...,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv,0,0.0,10.0
37,8/31/2021 4:21:23 PM,+31625317401 (0625317401),Q Wachtrij_DZH-ALmere (0501),00:00:08,almere.csv,0625317401,0625317401,25317401,2021-08-31 16:21:23,31/08/2021,...,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv,0,0.0,21.0
38,8/31/2021 4:55:54 PM,+31634199137 (0634199137),Q Wachtrij_DZH-ALmere (0501),00:00:07,almere.csv,0634199137,0634199137,34199137,2021-08-31 16:55:54,31/08/2021,...,0365369600,2021-08-31 16:00:00,65369600,2021-08-31 16:00:001579119595.0,365369600,Almere,almere.csv,0,0.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54033,9/29/2021 1:22:58 PM,+31611865036 (0611865036),Q Q_Anicura-Zoetermeer (1515),00:00:20,Zoetermeer.csv,0611865036,0611865036,11865036,2021-09-29 13:22:58,29/09/2021,...,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,0.0,0.0
54034,9/29/2021 1:28:57 PM,+31646351236 (0646351236),Q Q_Anicura-Zoetermeer (1515),00:00:04,Zoetermeer.csv,0646351236,0646351236,46351236,2021-09-29 13:28:57,29/09/2021,...,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,0.0,6.0
54035,9/29/2021 2:11:45 PM,+31626330463 (0626330463),Q Q_Anicura-Zoetermeer (1515),00:00:28,Zoetermeer.csv,0626330463,0626330463,26330463,2021-09-29 14:11:45,29/09/2021,...,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,0.0,49.0
54036,9/29/2021 2:20:39 PM,+31614768404 (0614768404),Q Q_Anicura-Zoetermeer (1515),00:00:12,Zoetermeer.csv,0614768404,0614768404,14768404,2021-09-29 14:20:39,29/09/2021,...,0793460214,2021-09-29 13:22:00,93460214,2021-09-29 13:22:001010637573.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,0.0,58.0


In [37]:
###Remove Duplicate rowidentifiers, by selecting the best match (minimum difference in minutes)
Google_Voip = Google_Voip.sort_values(["RowIdentifier", "min"], ascending = (False, True))
Google_Voip=Google_Voip.sort_values('min').drop_duplicates(subset='RowIdentifier', keep='first')

In [39]:
Google_Voip

,Call Time,Caller ID,Destination,Talking,file,Phone_x,Phone2,Phone_trim_x,Date_x,Date short,...,Phone_y,Date_y,Phone_trim_y,RowIdentifier,Telefoonnummer,Clinic,File name,timejumps,hours,min
51338,9/29/2021 5:16:54 PM,DKL-Sleeuwijk_31183301708:+31654380307 (065438...,Q Q_Sleeuwijk (1520),00:00:45,sleeuwijk.csv,0654380307,0654380307,54380307,2021-09-29 17:16:54,29/09/2021,...,0183301708,2021-09-29 17:16:00,83301708,2021-09-29 17:16:001500688525.0,31183301708,AniCura Sleeuwijk,sleeuwijk.csv,0,0.0,0.0
12007,9/21/2021 10:04:55 PM,+31618633465 (0618633465),Q Q_SPOED-MCD (1602),00:01:34,MCD.csv,0618633465,0618633465,18633465,2021-09-21 22:04:55,21/09/2021,...,31207400600,2021-09-21 22:04:00,07400600,2021-09-21 22:04:001808063902.0,31207400600,Anicura MCD,MCD.csv,0,0.0,0.0
51777,9/6/2021 8:58:56 AM,+31630983095 (0630983095),Q Q_Anicura-Zoetermeer (1515),00:00:10,Zoetermeer.csv,0630983095,0630983095,30983095,2021-09-06 08:58:56,06/09/2021,...,0793460214,2021-09-06 08:58:00,93460214,2021-09-06 08:58:0051961991.16,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,0.0,0.0
22079,9/6/2021 8:47:42 AM,DKL-Sleeuwijk_31183301708:+31623187330 (062318...,Q Q_Sleeuwijk (1520),unanswered,sleeuwijk.csv,0623187330,0623187330,23187330,2021-09-06 08:47:42,06/09/2021,...,0183301708,2021-09-06 08:47:00,83301708,2021-09-06 08:47:001784557653.0,31183301708,AniCura Sleeuwijk,sleeuwijk.csv,0,0.0,0.0
43316,9/22/2021 8:33:56 AM,DKL-Sleeuwijk_31183301708:+31622240079 (062224...,Q Q_Sleeuwijk (1520),00:00:45,sleeuwijk.csv,0622240079,0622240079,22240079,2021-09-22 08:33:56,22/09/2021,...,0183301708,2021-09-22 08:33:00,83301708,2021-09-22 08:33:00471679125.2,31183301708,AniCura Sleeuwijk,sleeuwijk.csv,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,9/16/2021 5:21:38 PM,+31622363154 (0622363154),Q Wachtrij_DZH-ALmere (0501),00:00:29,almere.csv,0622363154,0622363154,22363154,2021-09-16 17:21:38,16/09/2021,...,0365369600,2021-09-16 16:48:00,65369600,2021-09-16 16:48:001128937986.0,365369600,Almere,almere.csv,0,0.0,33.0
53614,9/20/2021 7:53:36 PM,+31703871096 (0703871096),Q Q_Anicura-Zoetermeer (1515),00:00:20,Zoetermeer.csv,0703871096,0703871096,03871096,2021-09-20 19:53:36,20/09/2021,...,0793460214,2021-09-20 19:12:00,93460214,2021-09-20 19:12:001227323731.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,0.0,41.0
3054,9/13/2021 9:00:28 AM,+31625387109 (0625387109),Q Q_Hoogvliet (1615),00:00:06,Hoogvliet.csv,0625387109,0625387109,25387109,2021-09-13 09:00:28,13/09/2021,...,0102950530,2021-09-13 08:17:00,02950530,2021-09-13 08:17:0026069123.16,31102950530,AniCura Dierenkliniek Hoogvliet,Hoogvliet.csv,0,0.0,43.0
51691,9/2/2021 5:28:21 PM,+31624097022 (0624097022),Q Q_Anicura-Zoetermeer (1515),00:00:22,Zoetermeer.csv,0624097022,0624097022,24097022,2021-09-02 17:28:21,02/09/2021,...,0793460214,2021-09-02 16:44:00,93460214,2021-09-02 16:44:001053323561.0,31793460214,AniCura Dierenkliniek Zoetermeer,Zoetermeer.csv,0,0.0,44.0


In [68]:
Google_Voip2.to_excel(r"C:\Users\luisgarc0001\Documents\Projects\Marketing Analytics\Google_Voip2.xlsx")
Google_Voip.to_excel(r"C:\Users\luisgarc0001\Documents\Projects\Marketing Analytics\Google_Voip.xlsx")